In [64]:
import pandas as pd
import requests

In [65]:

lat = 59.9333
lon = 10.7166

url = f"https://api.met.no/weatherapi/nowcast/2.0/complete?lat={lat}&lon={lon}"
headers = {"User-Agent": "WeatherTest/1.0 skybert@gmail.com"}

response = requests.get(url, headers=headers)
result = {}

if response.status_code == 200:
    result = response.json()
else:
    print(f"Error: {response.status_code}")
    print(response.text)

In [68]:
import pytz
from datetime import datetime

def convert_time_stamp(timestamp_str):
    # Deinerer tidssonen for pytz
    utc_timezone = pytz.timezone('UTC')

    # Setter tidssonen vi vil ha det i
    local_timezone = pytz.timezone('Europe/Copenhagen')  # Replace 'Your/Timezone' with your timezone

    # Konverterer stringen til et datetime objekt
    timestamp_utc = datetime.strptime(timestamp_str, '%Y-%m-%dT%H:%M:%SZ')

    # Lokaliserer strengen til utc
    timestamp_utc = utc_timezone.localize(timestamp_utc)

    # Konverterer til vår lokale tidssone som vi satte før
    timestamp_local = timestamp_utc.astimezone(local_timezone)

    # Formatterer strengen slik at den er mer lesbar og gir mening for oss
    local_timestamp_str = timestamp_local.strftime('%Y-%m-%d %H:%M:%S %Z')

    return local_timestamp_str

In [69]:
print(result)

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [10.7166, 59.9333, 55]}, 'properties': {'meta': {'updated_at': '2024-02-12T17:18:52Z', 'units': {'air_temperature': 'celsius', 'precipitation_amount': 'mm', 'precipitation_rate': 'mm/h', 'relative_humidity': '%', 'wind_from_direction': 'degrees', 'wind_speed': 'm/s', 'wind_speed_of_gust': 'm/s'}, 'radar_coverage': 'ok'}, 'timeseries': [{'time': '2024-02-12T17:20:00Z', 'data': {'instant': {'details': {'air_temperature': -3.6, 'precipitation_rate': 0.4, 'relative_humidity': 82.4, 'wind_from_direction': 37.4, 'wind_speed': 3.7, 'wind_speed_of_gust': 7.3}}, 'next_1_hours': {'summary': {'symbol_code': 'snow'}, 'details': {'precipitation_amount': 0.4}}}}, {'time': '2024-02-12T17:25:00Z', 'data': {'instant': {'details': {'precipitation_rate': 0.4}}}}, {'time': '2024-02-12T17:30:00Z', 'data': {'instant': {'details': {'precipitation_rate': 0.4}}}}, {'time': '2024-02-12T17:35:00Z', 'data': {'instant': {'details': {'precipitation_ra

In [ ]:
# Lage variabler med dictonaries med relevant informasjon

updated_at = result["properties"]["meta"]["updated_at"]
units_dict = result["properties"]["meta"]["units"]


In [ ]:
units_measured_list = result["properties"]["timeseries"]

In [ ]:
# Lage noe her som tar tiden og konverterer i formatet jeg har satt nedenfor også joiner det sammen

for i in units_measured_list:
    print(i)

In [ ]:
units_dict

In [ ]:
unit_df = pd.DataFrame.from_dict(units_dict, orient='index', columns=['Unit'])
unit_df.index.name = 'Attribute'

In [ ]:
unit_df

In [ ]:

converted_time = convert_time_stamp(updated_at)
print(f"Sist oppdatert at {converted_time}")